# Temperature and Humidity Prediction using Pre-trained Model

This notebook uses the pre-trained model to predict temperature and humidity using other air quality features.
Changes:
- Uses pre-trained model instead of training new ones
- Uses project configuration and utilities
- Updated paths to use project structure

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import tensorflow as tf

# Add project root to path for imports
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.append(project_root)

# Import project modules
from src.config import DATA_PATH, MODEL_SAVE_PATH, RESULTS_PATH
from src.data_preprocessing import DataPreprocessor
from src.predict import predict_temp_hum, load_optimized_model
from src.visualization import Visualizer

ModuleNotFoundError: No module named 'utils'

## 1. Load Pre-trained Model and Data

In [ ]:
# Load the optimized model
model_path = os.path.join(MODEL_SAVE_PATH, 'best_model.h5')
best_model = load_optimized_model(model_path)

# Load and prepare data
df = pd.read_csv(DATA_PATH)

# Initialize preprocessor
preprocessor = DataPreprocessor()

# Select features for prediction (excluding temperature and humidity)
reg_features = ['CO2', 'TVOC', 'PM10', 'PM2.5', 'CO', 'LDR', 'O3']
X_reg = df[reg_features]
y_temp = df['Temp']
y_hum = df['Hum']

# Preprocess data
X_reg_scaled = preprocessor.normalize_features(X_reg, None)[0]

## 2. Make Temperature and Humidity Predictions

In [ ]:
# Make predictions using the pre-trained model
temp_pred, hum_pred = predict_temp_hum(best_model, X_reg_scaled)

# Calculate metrics for temperature predictions
temp_metrics = {
    'MSE': mean_squared_error(y_temp, temp_pred),
    'RMSE': np.sqrt(mean_squared_error(y_temp, temp_pred)),
    'R²': r2_score(y_temp, temp_pred),
    'MAE': mean_absolute_error(y_temp, temp_pred)
}

# Calculate metrics for humidity predictions
hum_metrics = {
    'MSE': mean_squared_error(y_hum, hum_pred),
    'RMSE': np.sqrt(mean_squared_error(y_hum, hum_pred)),
    'R²': r2_score(y_hum, hum_pred),
    'MAE': mean_absolute_error(y_hum, hum_pred)
}

print("\nTemperature Prediction Metrics:")
for metric, value in temp_metrics.items():
    print(f"{metric}: {value:.4f}")

print("\nHumidity Prediction Metrics:")
for metric, value in hum_metrics.items():
    print(f"{metric}: {value:.4f}")

## 3. Visualize Results

In [ ]:
# Create visualizer instance
viz = Visualizer()

# Plot temperature predictions
plt.figure(figsize=(15, 5))

plt.subplot(1, 2, 1)
viz.plot_regression_results(y_temp, temp_pred, 'Temperature Prediction')

plt.subplot(1, 2, 2)
viz.plot_regression_results(y_hum, hum_pred, 'Humidity Prediction')

plt.tight_layout()
plt.show()

## 4. Save Results

In [ ]:
# Save metrics
metrics = {
    'Temperature': temp_metrics,
    'Humidity': hum_metrics
}

import pickle
metrics_path = os.path.join(RESULTS_PATH, 'temp_hum_prediction_metrics.pkl')
with open(metrics_path, 'wb') as f:
    pickle.dump(metrics, f)

print(f"Metrics saved to {metrics_path}")